# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Function for Pandas DataFrams

In [2]:
def create_frame(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

# file_path_list is declared as an empty list. It will be used to store the file paths found during the iteration.    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))    
    #print(file_path_list)


**Note:**

In Jupyter Notebook, if you execute the code cell by cell, the variable file_path_list will retain its 
value and be accessible outside the loop even if it was defined within the loop.

However, when running a Python script in a regular Python file, the scoping rules are different. 
Variables defined within a loop are generally limited to the scope of that loop. Once the loop finishes 
executing, the variable is no longer accessible outside the loop.

In [4]:
#Checking file_path_list
for f in file_path_list:
    print(f)

/home/jarminet/Desktop/Udacity Data Engineering/Project1-Cassandra/Data-Modeling-with-Apache-Cassandra/event_data/2018-11-22-events.csv
/home/jarminet/Desktop/Udacity Data Engineering/Project1-Cassandra/Data-Modeling-with-Apache-Cassandra/event_data/2018-11-16-events.csv
/home/jarminet/Desktop/Udacity Data Engineering/Project1-Cassandra/Data-Modeling-with-Apache-Cassandra/event_data/2018-11-17-events.csv
/home/jarminet/Desktop/Udacity Data Engineering/Project1-Cassandra/Data-Modeling-with-Apache-Cassandra/event_data/2018-11-27-events.csv
/home/jarminet/Desktop/Udacity Data Engineering/Project1-Cassandra/Data-Modeling-with-Apache-Cassandra/event_data/2018-11-09-events.csv
/home/jarminet/Desktop/Udacity Data Engineering/Project1-Cassandra/Data-Modeling-with-Apache-Cassandra/event_data/2018-11-01-events.csv
/home/jarminet/Desktop/Udacity Data Engineering/Project1-Cassandra/Data-Modeling-with-Apache-Cassandra/event_data/2018-11-28-events.csv
/home/jarminet/Desktop/Udacity Data Engineering/

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
print("Number of rows in CSV File (Column's name included):")
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

Number of rows in CSV File (Column's name included):
6821


# Part II - Now we are ready to work with the CSV file titled <font color=red>./event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1']) 

# To establish connection and begin executing queries, need a session
try:
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [8]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Query 1: 

Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

`SELECT artist, song, length FROM music_library WHERE sessionId=338 AND itemInSession=4"` 

The table music_app will be created with PRIMARY KEYS itemInSession and sessionId based on the WHERE clause needed for Query1, and after looking at the data we can see that these two columns together can identify uniquely each row.

#### Table creation

In [10]:
session.execute("DROP TABLE IF EXISTS song_details")
query = "CREATE TABLE IF NOT EXISTS song_details "
query = query + "(artist text, itemInSession int, length float, sessionId int, song text, "
query = query + "PRIMARY KEY ((itemInSession,sessionId)))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Data insertion

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_details (artist, itemInSession, length, sessionId, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"        
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### SELECT to verify that the data have been inserted into each table

In [12]:
session.row_factory = create_frame
rows = session.execute("SELECT COUNT(*) FROM song_details")

print("We expect 6,820 rows inserted in song_details")

df = rows._current_rows 
df

We expect 6,820 rows inserted in song_details


count
0   6820

#### Query1 execution

In [13]:
session.row_factory = create_frame
query = "SELECT artist, song, length FROM song_details WHERE sessionId=338 AND itemInSession=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = rows._current_rows 
df

artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312

## Query 2: 

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

`SELECT artist, song, firstName, lastName FROM music_app2 WHERE sessionId='182' AND userId='10'`

A new table (music_app2) will be created.  In this case, we need itemInSession at the end of our Clustering Columns, because we need the result sorted by this column.

#### Table creation

In [14]:
session.execute("DROP TABLE IF EXISTS artist_details")
query = "CREATE TABLE IF NOT EXISTS artist_details"
query = query + "(artist text, firstName text, itemInSession int, lastName text, sessionId int, song text, userId int, "
query = query + "PRIMARY KEY ((sessionId, userId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e) 

#### Data Insertion

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_details (artist, firstName, itemInSession, lastName, sessionId, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"        
        session.execute(query, (line[0], line[1], int(line[3]), line[4], int(line[8]), line[9], int(line[10])))

#### SELECT to verify that the data have been inserted into each table music_app2

In [16]:
session.row_factory = create_frame
rows = session.execute("SELECT COUNT(*) FROM artist_details")

print("We expect 6,820 rows inserted in artist_details")

df = rows._current_rows 
df

We expect 6,820 rows inserted in artist_details


count
0   6820

#### Query2 execution

In [17]:
query = "SELECT artist, song, firstName, lastName FROM artist_details WHERE sessionId=182 AND userId=10"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = rows._current_rows 
df

artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz

## Query 3: 

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

`SELECT firstName, lastName FROM music_app3 WHERE song='All Hands Against His Own'`

#### Table creation and data insertion

In [52]:
session.execute("DROP TABLE IF EXISTS user_details")
query = "CREATE TABLE IF NOT EXISTS user_details"
query = query + "(firstName text, lastName text, song text, userId int, "
query = query + "PRIMARY KEY (song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_details (firstName, lastName, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"       
        session.execute(query, (line[1], line[4], line[9], int(line[10])))
                    

#### SELECT to verify that the data have been inserted into each table music_app3

In [53]:
rows = session.execute("SELECT COUNT(*) FROM user_details")

df = rows._current_rows 
df

count
0   6618

#### Query3 execution

In [54]:
query = "SELECT firstName, lastName FROM user_details WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = rows._current_rows 
df

firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson

### Drop the tables before closing out the sessions

In [93]:
session.execute("DROP TABLE IF EXISTS song_details")
session.execute("DROP TABLE IF EXISTS artist_details")
session.execute("DROP TABLE IF EXISTS user_details")

### Close the session and cluster connection¶

In [153]:
session.shutdown()
cluster.shutdown()